In [ ]:
import yfinance as yf
import pandas as pd
import ta
import time
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer



# Baixar os últimos 30 dias de dados da Apple
ticker = "AAPL"
def coletar_dados():
    dados = yf.download(ticker, period="30d", interval="1d")

    # Garantir que "Close" seja uma série 1D
    dados["Close"] = dados["Close"].astype(float).squeeze()

    # Calcular a Média Móvel Simples (SMA) de 10 e 20 dias
    dados["SMA_10"] = dados["Close"].rolling(window=10).mean()
    dados["SMA_20"] = dados["Close"].rolling(window=20).mean()

    # Calcular o RSI (Relative Strength Index) de 14 dias
    rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
    dados["RSI"] = rsi_indicator.rsi()

    # Calcular as Bandas de Bollinger (20 dias)
    bollinger = ta.volatility.BollingerBands(close=dados["Close"].squeeze(), window=20)
    dados["Bollinger_High"] = bollinger.bollinger_hband()
    dados["Bollinger_Low"] = bollinger.bollinger_lband()
    return dados

def gerar_recomendacao(df):
    if len(df) < 20:
        return "Dados insuficientes para análise."

    ult_linha = df.iloc[-1]  # Última linha corretamente indexada
    recomendacoes = []

    # 🔹 Verificando se "RSI" existe e não é NaN antes da análise
    if "RSI" in df.columns:
        rsi_valor = ult_linha["RSI"].values[0] if isinstance(ult_linha["RSI"], pd.Series) else ult_linha["RSI"]
        print("Última linha:", ult_linha)  # Debugging: print the last row
        print("Valor do RSI:", rsi_valor)  # Debugging: print the RSI value
        
        if isinstance(rsi_valor, (float, int)) and not pd.isna(rsi_valor):  # Verificando se o valor não é NaN
            if rsi_valor < 30:
                recomendacoes.append("RSI indica COMPRA (ativo sobrevendido).")
            elif rsi_valor > 70:
                recomendacoes.append("RSI indica VENDA (ativo sobrecomprado).")
            
    # 🔹 Verificando cruzamento de médias móveis
    if len(df) > 1:
        prev_linha = df.iloc[-2]
        sma_10_prev = prev_linha["SMA_10"].values[0] if isinstance(prev_linha["SMA_10"], pd.Series) else prev_linha["SMA_10"]
        sma_20_prev = prev_linha["SMA_20"].values[0] if isinstance(prev_linha["SMA_20"], pd.Series) else prev_linha["SMA_20"]
        sma_10_ult = ult_linha["SMA_10"].values[0] if isinstance(ult_linha["SMA_10"], pd.Series) else ult_linha["SMA_10"]
        sma_20_ult = ult_linha["SMA_20"].values[0] if isinstance(ult_linha["SMA_20"], pd.Series) else ult_linha["SMA_20"]

        if sma_10_prev < sma_20_prev and sma_10_ult > sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica COMPRA.")
        elif sma_10_prev > sma_20_prev and sma_10_ult < sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica VENDA.")

    # 🔹 Analisando Bandas de Bollinger
    if "Close" in df.columns and "Bollinger_Low" in df.columns and "Bollinger_High" in df.columns:
        close_valor = ult_linha["Close"].values[0] if isinstance(ult_linha["Close"], pd.Series) else ult_linha["Close"]
        bollinger_low = ult_linha["Bollinger_Low"].values[0] if isinstance(ult_linha["Bollinger_Low"], pd.Series) else ult_linha["Bollinger_Low"]
        bollinger_high = ult_linha["Bollinger_High"].values[0] if isinstance(ult_linha["Bollinger_High"], pd.Series) else ult_linha["Bollinger_High"]

        if close_valor < bollinger_low:
            recomendacoes.append("Banda de Bollinger indica COMPRA (preço abaixo da banda inferior).")
        elif close_valor > bollinger_high:
            recomendacoes.append("Banda de Bollinger indica VENDA (preço acima da banda superior).")
    
    return "\n".join(recomendacoes) if recomendacoes else "Nenhuma recomendação clara no momento."


def coletar_noticias(ticker):
    url = f"https://newsapi.org/v2/everything?q={ticker}&apiKey=1aaf7f3923f543fe89ae1dd8688d0a1c"
    resposta = requests.get(url)
    dados = resposta.json()
    
    artigos = [artigo["title"] + " " + artigo["description"] for artigo in dados["articles"]]
    return artigos[:5]  # Pegamos apenas as 5 notícias mais recentes

noticias = coletar_noticias("Apple")
print(noticias)
nltk.download("vader_lexicon")

def analisar_sentimento_vader(noticias):
    sia = SentimentIntensityAnalyzer()
    scores = [sia.polarity_scores(noticia)["compound"] for noticia in noticias]

    media_sentimento = sum(scores) / len(scores) if scores else 0

    if media_sentimento > 0.05:
        return "Positivo"
    elif media_sentimento < -0.05:
        return "Negativo"
    else:
        return "Neutro"

sentimento = analisar_sentimento_vader(noticias)
print("Sentimento das notícias:", sentimento)
# Supondo que 'dados' seja o seu DataFrame
# while True:
print("\n🔄 Atualizando análise...")
dados = coletar_dados()
def gerar_recomendacao_final(indicador, sentimento):
    if indicador == "COMPRA" and sentimento == "Positivo":
        return "✅ Forte recomendação de COMPRA!"
    elif indicador == "VENDA" and sentimento == "Negativo":
        return "🚨 Forte recomendação de VENDA!"
    elif indicador == "COMPRA" and sentimento == "Negativo":
        return "⚠ Compra arriscada (notícias negativas)."
    elif indicador == "VENDA" and sentimento == "Positivo":
        return "⚠ Venda arriscada (notícias positivas)."
    else:
        return "🤷 Nenhuma recomendação clara."

indicador_tecnico = "COMPRA"  # Simulação do seu indicador técnico
recomendacao_final = gerar_recomendacao_final(indicador_tecnico, sentimento)


if dados is not None:
    gerar_recomendacao(dados)
    print("\n📊 Recomendação baseada nos indicadores:\n")
    print(recomendacao_final)
else:
    print("⚠ Erro ao coletar dados. Tentando novamente...")


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 🔹 Coletar dados do Yahoo Finance
ticker = "AAPL"
dados = yf.download(ticker, period="1y", interval="1d")

# 🔹 Selecionar apenas o preço de fechamento
fechamento = dados["Close"].values.reshape(-1, 1)

# 🔹 Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
fechamento_normalizado = scaler.fit_transform(fechamento)

# 🔹 Criar sequências para a LSTM (usando 60 dias anteriores para prever o próximo)
def criar_sequencias(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60  # Usar 60 dias para prever o próximo dia
X, y = criar_sequencias(fechamento_normalizado, seq_length)

# 🔹 Dividir os dados em treino e teste (80% treino, 20% teste)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 🔹 Construir o modelo LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# 🔹 Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# 🔹 Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# 🔹 Fazer previsões
y_pred = model.predict(X_test)

# 🔹 Reverter a normalização para ver os preços reais
y_pred_real = scaler.inverse_transform(y_pred)
y_test_real = scaler.inverse_transform(y_test)

# 🔹 Visualizar os resultados
plt.figure(figsize=(12, 6))
plt.plot(y_test_real, label="Preço Real")
plt.plot(y_pred_real, label="Preço Previsto", linestyle="dashed")
plt.legend()
plt.show()

# 🔹 Determinar sinais de COMPRA e VENDA
compras = []
vendas = []
for i in range(1, len(y_pred_real)):
    if y_pred_real[i] > y_pred_real[i - 1]:  
        compras.append(i)
    elif y_pred_real[i] < y_pred_real[i - 1]:  
        vendas.append(i)

print("🔹 Sinais de compra:", compras)
print("🔹 Sinais de venda:", vendas)


[*********************100%***********************]  1 of 1 completed

Epoch 1/50



c:\Users\CarlosEduardoPiacent\Downloads\Tcc\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(16,), dtype=float32). Expected shape (None, 60, 1), but input has incompatible shape (16,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(16,), dtype=float32)
  • training=True
  • mask=None

In [ ]:
import mysql.connector
import yfinance as yf
import ta
import requests
import yaml
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from datetime import datetime


def carregar_config():
    with open("config.yml", "r") as file:
        config = yaml.safe_load(file)
    return config["database"]

# Função para se conectar ao banco de dados
def conectar_db():
    config = carregar_config()
    return mysql.connector.connect(
        host=config["host"],
        user=config["user"],
        password=config["password"],
        database=config["database"]
    )

def salvar_dados_acao(df):
    conn = conectar_db()
    cursor = conn.cursor()


    for index, row in df.iterrows():
        # Certifique-se de que 'ticker' está definido e é uma string
        ticker_value = str(ticker)  # A variável 'ticker' precisa estar definida em algum lugar
        date_value = row.name.strftime('%Y-%m-%d')  # Data como string
        
        # Directly access the value using row[]
        close_value = None if pd.isna(row["Close"]) else float(row["Close"])
        sma_10_value = None if pd.isna(row["SMA_10"]) else float(row["SMA_10"])
        sma_20_value = None if pd.isna(row["SMA_20"]) else float(row["SMA_20"])
        rsi_value = None if pd.isna(row["RSI"]) else float(row["RSI"])
        bollinger_high_value = None if pd.isna(row["Bollinger_High"]) else float(row["Bollinger_High"])
        bollinger_low_value = None if pd.isna(row["Bollinger_Low"]) else float(row["Bollinger_Low"])

        # Verificando se as variáveis estão sendo definidas corretamente
        print(f"Ticker: {ticker_value}, Date: {date_value}, Close: {close_value}, SMA_10: {sma_10_value}, SMA_20: {sma_20_value}, RSI: {rsi_value}, Bollinger_High: {bollinger_high_value}, Bollinger_Low: {bollinger_low_value}")
        
        cursor.execute("""
            INSERT INTO stocks_data (ticker, date, close, sma_10, sma_20, rsi, bollinger_high, bollinger_low)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (ticker_value, date_value, close_value, sma_10_value, sma_20_value, rsi_value, bollinger_high_value, bollinger_low_value))
    
    conn.commit()
    cursor.close()
    conn.close()

# Função para salvar análise de sentimento das notícias
def salvar_analise_sentimento(ticker, sentiment, articles):
    conn = conectar_db()
    cursor = conn.cursor()

    for article in articles:
        cursor.execute("""
            INSERT INTO news_sentiment_analysis (ticker, date, sentiment, article_title)
            VALUES (%s, %s, %s, %s)
        """, (ticker, datetime.now().strftime('%Y-%m-%d'), sentiment, article))

    conn.commit()
    cursor.close()
    conn.close()

# Função para coletar notícias e analisar sentimento
def coletar_noticias(ticker):
    url = f"https://newsapi.org/v2/everything?q={ticker}&apiKey=1aaf7f3923f543fe89ae1dd8688d0a1c"
    resposta = requests.get(url)
    dados = resposta.json()

    artigos = [artigo["title"] + " " + artigo["description"] for artigo in dados["articles"]]
    return artigos[:5]  # Pegamos apenas as 5 notícias mais recentes

# Função para analisar o sentimento das notícias
def analisar_sentimento_vader(noticias):
    sia = SentimentIntensityAnalyzer()
    scores = [sia.polarity_scores(noticia)["compound"] for noticia in noticias]

    media_sentimento = sum(scores) / len(scores) if scores else 0

    if media_sentimento > 0.05:
        return "Positivo"
    elif media_sentimento < -0.05:
        return "Negativo"
    else:
        return "Neutro"

# Função para coletar dados financeiros do Yahoo
def coletar_dados():
    dados = yf.download(ticker, period="30d", interval="1d")
    dados["SMA_10"] = dados["Close"].rolling(window=10).mean()
    dados["SMA_20"] = dados["Close"].rolling(window=20).mean()
    rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
    dados["RSI"] = rsi_indicator.rsi()
    bollinger = ta.volatility.BollingerBands(close=dados["Close"].squeeze(), window=20)
    dados["Bollinger_High"] = bollinger.bollinger_hband()
    dados["Bollinger_Low"] = bollinger.bollinger_lband()
    return dados

# Função principal
def main():
    ticker = "AAPL"

    # Coletando dados financeiros
    dados = coletar_dados()
    salvar_dados_acao(dados)

    # Coletando e analisando notícias
    noticias = coletar_noticias("Apple")
    sentimento = analisar_sentimento_vader(noticias)
    salvar_analise_sentimento(ticker, sentimento, noticias)

    print("Análise de sentimentos e dados financeiros salvos com sucesso!")

# Executando o código
main()
